In [1]:
import numpy as np
import os
import torch
from misc.example_helper import *
import importlib
import processing
import models
%load_ext autoreload
%autoreload 2
importlib.reload(processing)
importlib.reload(models)
from processing.parametric_data_manager import ParametricSHREDDataManager
from models.shred_models import SHRED

In [2]:
dataset = np.load('kuramoto_sivashinsky\KuramotoSivashinsky_data.npz')
data = dataset['u'] # shape (500, 201, 100)
mu = dataset['mu'] # shape (500, 201, 2)

# # Add data to manager (with sensors)
# ni_3D = np.load('data4fields_npz/ni_3D.npz')
# ni_3D = ni_3D[ni_3D.files[0]]
# mobile_sensors = [
#     perimeter_walk(height = ni_3D.shape[1], width = ni_3D.shape[2], timesteps = ni_3D.shape[0], clockwise=True),
#     perimeter_walk(height = ni_3D.shape[1], width = ni_3D.shape[2], timesteps = ni_3D.shape[0], clockwise=False)
# ]
# manager.add(
#     data=ni_3D,
#     random_sensors=2,
#     stationary_sensors=[(7, 9), (0,0)],
#     mobile_sensors=mobile_sensors
# )

# print(f"Processed: {filename}")

In [3]:
# Generate a list of indices that alternates back and forth from 0 to 99 until the list length is 201
indices = []

# Start generating indices
while len(indices) < 201:
    indices += list(range(100))  # Add 0 to 99
    indices += list(range(99, -1, -1))  # Add 99 to 0

# Truncate the list to ensure the length is exactly 201
indices = indices[:201]

print(len(indices))


201


In [5]:
mobile_sensors = forward_backward_walk(start=0, end = data.shape[2], timesteps=data.shape[1])

In [7]:
len(mobile_sensors)

201

In [12]:
import os

# Initialize ParametricSHREDDataManager
manager = ParametricSHREDDataManager(
    lags = 20,
    train_size = 0.7,
    val_size = 0.15,
    test_size = 0.15,
    scaling = "minmax",
    compression = 20,
    time=np.arange(0, 2000),
    )

# Add data to manager (with sensors)
dataset = np.load('kuramoto_sivashinsky\KuramotoSivashinsky_data.npz')
data = dataset['u'] # shape (500, 201, 100)
mu = dataset['mu'] # shape (500, 201, 2)

# mobile_sensors = forward_backward_walk(start=0, end = data[2], timesteps=data[1])
# mobile_sensors = [
#     perimeter_walk(height = ni_3D.shape[1], width = ni_3D.shape[2], timesteps = ni_3D.shape[0], clockwise=True),
#     perimeter_walk(height = ni_3D.shape[1], width = ni_3D.shape[2], timesteps = ni_3D.shape[0], clockwise=False)
# ]
manager.add(
    data=data,
    random_sensors=2,
    stationary_sensors=[(0,), (1,)],
    # mobile_sensors=mobile_sensors,
    params=mu,
)


train indices [ 52 475 367 260 251 459 350 301 454 307 148 299 391 187  47 112 189 234
 143  12 144  37 438 419 316 274  23 366 122 212 128 462 201 480 387 378
 288 377 215 317 484 333 348 105 415 386 488 275 290 327 192   0  80 395
 404 487 168 423 451 304 242 456 436 121 111 282 211  34 185  56  91 368
 173 345 478  90 388 405 401 106 457 413 418 262 305 222 425 232 263  38
 342 314 338 256 257  73 442  32 402 252 473  55   5  71 494 102 285  44
  57 337  25 155 433 435   8 294  33 409 363   9 110  31 334 369 125 117
 492 118 399 188 410 347  94 134  77 383  51 233 280 246 265 312  88 235
 474  20 127 254 331 170 427 351 225 283 360  36 167 141 230 499 460 156
 470  62 394 182 295 281  83 172 202 209 406 292 446 177  98 268 289 324
 214 147  30 149 266 444 176 253 483  66   2 108 380 146 272 113 287  75
 207  50 421 213  67 297 164 261 247 319 204 267 114 485 195 277 346 340
 320  70  29 339 160   7 379 205 371 440 359  79 308 385 140 107 417  81
 332  76 243  17  13 420 464 196 441 

ValueError: cannot reshape array of size 422100 into shape (35,2000,6)

In [19]:
# Non-Parametric Case - train/val/test split by timesteps
single_traj = np.load('data3fields_npz/ni_3D.npz')
single_traj = single_traj[single_traj.files[0]]
single_traj = np.transpose(single_traj, (2,0,1))
print('single_traj:', single_traj.shape)
manager = SHREDDataManager(lags = 20, train_size = 0.7, val_size = 0.15, test_size = 0.15, scaling = "minmax", compression = 20, reconstructor=True, forecastor=True)
manager.add(
    data = single_traj,
    random_sensors = 3,
    time=np.arange(0, 2000)
)
manager.add(
    data = single_traj,
    stationary_sensors= [(0,5), (5,0)],
    time=np.arange(0, 2000)
)

single_traj: (2000, 257, 256)
self.full_state_data flattened (2000, 65792)
n_components 20
V.shape (20, 65792)
compressed full_state_data: (2000, 20)
self.full_state_data pre transform (2000, 65792)
transformed_data_std_scale (2000, 65792)
transformed_data (2000, 20)
self.full_state_data flattened (2000, 65792)
n_components 20
V.shape (20, 65792)
compressed full_state_data: (2000, 20)
self.full_state_data pre transform (2000, 65792)
transformed_data_std_scale (2000, 65792)
transformed_data (2000, 20)


In [20]:
train_set, val_set, test_set = manager.preprocess()

In [21]:
train_set.forecaster.X.shape

torch.Size([1399, 21, 5])

In [22]:
train_set.forecaster.Y.shape

torch.Size([1399, 5])

In [23]:
print(val_set.forecaster.X.shape)
print(val_set.forecaster.Y.shape)

torch.Size([299, 21, 5])
torch.Size([299, 5])


In [24]:
print(test_set.forecaster.X.shape)
print(test_set.forecaster.Y.shape)

torch.Size([299, 21, 5])
torch.Size([299, 5])


In [32]:
print(test_set.forecaster.X[1,-1,:] == test_set.forecaster.Y[0,:])
print(val_set.forecaster.X[1,-1,:] == val_set.forecaster.Y[0,:])
print(train_set.forecaster.X[1,-1,:] == train_set.forecaster.Y[0,:])

tensor([True, True, True, True, True])
tensor([True, True, True, True, True])
tensor([True, True, True, True, True])


In [30]:
test_set.forecaster.Y[0,:]

tensor([0.4656, 0.4639, 0.7151, 0.5001, 0.3212])

In [25]:
shred = SHRED(sequence='LSTM', decoder='SDN')
shred.fit(train_set, val_set, num_epochs=10)


Fitting Reconstructor...


Epoch 10/10: 100%|██████████| 22/22 [00:00<00:00, 99.36batch/s, loss=0.00134, L2=0.058, val_loss=3.39e-5, val_L2=0.00923] 


input_size 5
output_size 5

Fitting Forecaster...


Epoch 10/10: 100%|██████████| 22/22 [00:00<00:00, 106.68batch/s, loss=0.0262, L2=0.311, val_loss=0.0216, val_L2=0.264]


(array([0.18292472, 0.04216024, 0.01591722, 0.00958682, 0.00560595,
        0.00432521, 0.00644511, 0.00749968, 0.00687222, 0.00922802],
       dtype=float32),
 array([0.31393322, 0.2736822 , 0.26062334, 0.25669444, 0.25526607,
        0.25716653, 0.25383165, 0.25632235, 0.2670341 , 0.26433328],
       dtype=float32))

In [5]:
# Non-Parametric Case - train/val/test split by timesteps
single_traj = np.load('data3fields_npz/ni_3D.npz')
single_traj = single_traj[single_traj.files[0]]
single_traj = np.transpose(single_traj, (2,0,1))
print('single_traj:', single_traj.shape)
manager = SHREDDataManager(lags = 20, train_size = 0.7, val_size = 0.15, test_size = 0.15, scaling = "minmax", compression = 20, reconstructor=True, forecastor=True)
manager.add(
    data = single_traj,
    random_sensors = 3,
    time=np.arange(0, 2000)
)
manager.add(
    data = single_traj,
    stationary_sensors= [(0,5), (5,0)],
    time=np.arange(0, 2000)
)

# multi_traj = np.stack((single_traj, single_traj, single_traj,single_traj), axis=0)

single_traj: (2000, 257, 256)
self.full_state_data flattened (2000, 65792)
n_components 20
V.shape (20, 65792)
compressed full_state_data: (2000, 20)
self.full_state_data pre transform (2000, 65792)
transformed_data_std_scale (2000, 65792)
transformed_data (2000, 20)
self.full_state_data flattened (2000, 65792)
n_components 20
V.shape (20, 65792)
compressed full_state_data: (2000, 20)
self.full_state_data pre transform (2000, 65792)
transformed_data_std_scale (2000, 65792)
transformed_data (2000, 20)
self.full_state_data flattened (2000, 65792)


KeyboardInterrupt: 

In [ ]:
# sample = np.load('data3fields_npz/ni_3D.npz')
# sample = sample[sample.files[0]]
# sensor_perimeter_walk_coordinates = perimeter_walk(height = sample.shape[0], width = sample.shape[1], timesteps = sample.shape[2], clockwise=True)
# multi_traj = np.stack((sample, sample, sample), axis=0)
# print('multi_traj:', multi_traj.shape)

In [ ]:
# multi_traj = np.transpose(multi_traj, (0,3,1,2))
# print('multi_traj:', multi_traj.shape)

In [ ]:
# Extract the frame to plot (shape: (257, 256))
trajectory_idx = 0
time_idx = 300
frame = multi_traj[trajectory_idx, time_idx, :, :]

# Plot the frame
plt.figure(figsize=(8, 6))
plt.imshow(frame, cmap="viridis", aspect="auto")
plt.colorbar(label="Intensity")
plt.title(f"Trajectory {trajectory_idx}, Frame {time_idx}")
plt.xlabel("Spatial Dimension 1")
plt.ylabel("Spatial Dimension 2")
plt.show()

In [ ]:
# multi_traj.shape

In [ ]:
multi_traj.shape

In [16]:
# Define dimensions
ntraj = multi_traj.shape[0]
ntime = multi_traj.shape[1]
nparams = 1

# Create the array
params = np.arange(ntraj).reshape(ntraj, 1, 1) * np.ones((1, ntime, nparams))

In [ ]:
# Parametric Case - train/val/test split by parameter

manager = ParametricSHREDDataManager(lags = 20, train_size = 0.5, val_size = 0.25, test_size = 0.25, scaling = "minmax", compression = 20)
manager.add(
    data=multi_traj,
    random_sensors=2,
    time=np.arange(0, 2000),
    params=params
)

In [22]:
train_set, val_set, test_set = manager.preprocess()

In [ ]:
train_set.forecaster == None

In [ ]:
train_set.reconstructor.Y.shape

In [ ]:
print('train X:',train_set.reconstructor.X.shape)
print('train Y:',train_set.reconstructor.Y.shape)

print('valid X:',val_set.reconstructor.X.shape)
print('valid Y:',val_set.reconstructor.Y.shape)

print('test X:',test_set.reconstructor.X.shape)
print('test Y:',test_set.reconstructor.Y.shape)

In [ ]:
multi_traj.shape

In [ ]:
manager = SHREDDataManager(parametric = False, lags = 20, train_size = 0.7, val_size = 0.15, test_size = 0.15, scaling = "minmax", compression = 20, reconstructor=True, forecastor=True)
manager.add(
    data = single_traj,
    random_sensors = 3,
    time=np.arange(0, 2000)
)
manager.add(
    data = single_traj,
    stationary_sensors= [(0,5), (5,0)],
    time=np.arange(0, 2000)
)

In [ ]:
# Parametric Case
manager = SHREDDataManager(lags=20, train_size=0.7, val_size=0.15, test_size=0.15, scaling="minmax", compression=20, )
manager.add(
    data=multi_traj,
    random_sensors=3,
    multi_traj=True,
    compression=20,
    scaling="minmax",
    time=np.arange(0, 2000)  # Assuming all files have 2000 timesteps
)

In [ ]:
train_set, val_set, test_set = manager.preprocess()

In [ ]:
# Parametric Case - train/val/test split by parameter

manager = ParametricSHREDDataManager(lags = 20, train_size = 0.5, val_size = 0.25, test_size = 0.25, scaling = "minmax", compression = 20)
manager.add(
    data=multi_traj,
    random_sensors=2,
    time=np.arange(0, 2000)
)

In [ ]:
train_set.reconstructor.X.shape

In [ ]:
# def validate_SHREDDataManager_init():
#     pass

# def validate_SHREDDataManger_add_field():
#     pass

# def validate_SHREDDataProcessor_init():
#     pass

In [ ]:
import os



# Initialize SHREDDataManager
manager = SHREDDataManager(lags=20, train_size=0.7, val_size=0.15, test_size=0.15)

# Folder containing .npz files
input_folder = "data3fields_npz"

# Process each .npz file
for filename in os.listdir(input_folder):
    if filename.endswith(".npz") and filename != 'ni_3D.npz':
        file_path = os.path.join(input_folder, filename)

        # Different fields need same number of timesteps or else some timesteps have 3 sensors while others have 5 sensors
        # hence different fields need same number of trajectories # else some trajectories have 3 sensors while others have 5 sensors, not gonna work.
        # number of sensors need to be consistent across all time and across all trajectories

        # manager.add_field(sensor_locs, n_params, n_trajectories???, ..., {id}) # since different fields need the same number of trajectories
        # actually not necessarily... if field 1 has 1000 timesteps (think of multiple trajects just as more timesteps, though not the same especially with lag generation),
        # then field 2 also needs 1000 timesteps for this to work right? Yes. Spatial shape can vary, but not timesteps I believe.

        # but sensor locs and n_params shouldn't be in add_field because it might not have those (want to reconstruct a field we aren't measuring from a field we are measuring)
        # though time and n_trajectories should remain constant across all fields.

        # maybe:
        # allow user to create SHREDDataProcessor object, add trajectories needed, then put into Manager. WAIT WHAT ABOUT TRAIN/VALID/TEST indices?
        # 

        # manager.{id}.add_trajectory()
        # BADDDDD SEE NOTES AND PHOTOS

        # Add the file to the SHREDDataManager
        manager.add_field(
            file_path=[file_path, file_path, file_path], # each element is a trajectory. Manager ensures all trajectories are number of timesteps, and all fields have same number of trajectories
            compression=20,
            scaling="minmax",
            time=np.arange(0, 2000)  # Assuming all files have 2000 timesteps
        )
        print(f"Processed: {filename}")

filename = 'ni_3D.npz'
file_path = os.path.join(input_folder, filename)

manager.add_field(
    file_path=[file_path, file_path, file_path],
    random_sensors=3,
    stationary_sensors=(7, 9),  # Example coordinates
    mobile_sensors=sensor_perimeter_walk_coordinates,
    compression=20,
    scaling="minmax",
    time=np.arange(0, 2000)  # Assuming all files have 2000 timesteps
)

In [51]:
train_set, val_set, test_set = manager.preprocess()

In [ ]:
train_set.reconstructor.X.shape[0] + val_set.reconstructor.X.shape[0] + test_set.reconstructor.X.shape[0]

In [ ]:
# should be (1400, 21, 5)
train_set.forecaster.X[0,:,:]

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
shred = models.SHRED(train_set.reconstructor.X.shape[-1], train_set.reconstructor.Y.shape[-1], hidden_size=64, hidden_layers=2, l1=350, l2=400, dropout=0.1).to(device)
validation_errors = models.fit(shred, train_set.reconstructor, val_set.reconstructor, batch_size=64, num_epochs=1000, lr=1e-3, verbose=True, patience=40)

In [ ]:
prediction = shred(test_set.reconstructor.X).detach().cpu().numpy()
prediction.shape